In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import numpy as np

In [2]:
songs = pd.read_csv("songs.csv")

In [3]:
songs.head()

,track,artist
0,"""Yuck!"" (featuring Lil Wayne )",2 Chainz
1,"""Crack""",2 Chainz
2,"""Dope Peddler""",2 Chainz
3,""" No Lie "" (featuring Drake )",2 Chainz
4,""" Birthday Song "" (featuring Kanye West )",2 Chainz


In [65]:
def clean_track(track):
    s = track
    s = re.sub(r'\(([^\)]+)\)','',s)
    
    s= re.sub(r'\"','',s )
    s = re.sub(r'\[.*?\']*','',s)
    s = re.sub(r'\'','',s)
    s = re.sub(r'--','',s)
    s = s.strip()
    s = s.replace(" ","-")
    s=s.lower()
    return s

In [66]:
songs["genius_track"] = songs["track"].apply(clean_track)

In [67]:
songs

,track,artist,genius_track,genius_artist
0,"""Yuck!"" (featuring Lil Wayne )",2 Chainz,yuck!,2-chainz
1,"""Crack""",2 Chainz,crack,2-chainz
2,"""Dope Peddler""",2 Chainz,dope-peddler,2-chainz
3,""" No Lie "" (featuring Drake )",2 Chainz,no-lie,2-chainz
4,""" Birthday Song "" (featuring Kanye West )",2 Chainz,birthday-song,2-chainz
5,""" I'm Different """,2 Chainz,im-different,2-chainz
6,"""Extremely Blessed"" (featuring The-Dream )",2 Chainz,extremely-blessed,2-chainz
7,"""I Luv Dem Strippers"" (featuring Nicki Mina...",2 Chainz,i-luv-dem-strippers,2-chainz
8,"""Stop Me Now"" (featuring Dolla Boy)",2 Chainz,stop-me-now,2-chainz
9,"""Money Machine""",2 Chainz,money-machine,2-chainz


In [6]:
def genius_artist(name):
    name =name[0]+name[1:].lower()
    name= name.replace(" ","-")
    return name

In [7]:
genius_artist("2 Chainz")

'2-chainz'

In [8]:
songs["genius_artist"] = songs["artist"].apply(genius_artist)

In [58]:
def get_lyrics(genius_artist,genius_track):
    base = 'https://genius.com/'
    url = base+genius_artist+"-"+genius_track+"-lyrics"
    page = requests.get(url)
    soup = BeautifulSoup(page.text,'html.parser')
    try:
        p = soup.find_all("p")
        s = p[0].text
        s = re.sub(r'\[.*?\']','',s)
        s = re.sub(r'--','',s)
        s = s.split("\n")
        s = [ x for x in s if x != ""]
        return s
    except:
        return "error"

In [59]:
soup = get_lyrics(songs["genius_artist"][971], songs["genius_track"][971])

In [60]:
soup

["    Sorry, we didn't mean for that to happen!", '  ']

In [11]:
songs.head()

,track,artist,genius_track,genius_artist
0,"""Yuck!"" (featuring Lil Wayne )",2 Chainz,yuck,2-chainz
1,"""Crack""",2 Chainz,crack,2-chainz
2,"""Dope Peddler""",2 Chainz,dope-peddler,2-chainz
3,""" No Lie "" (featuring Drake )",2 Chainz,no-lie,2-chainz
4,""" Birthday Song "" (featuring Kanye West )",2 Chainz,birthday-song,2-chainz


In [85]:
songs.shape

(27050, 4)

In [86]:
songs2 = songs[20001:27050]

In [88]:
songs2["lyrics"] = songs2.apply(lambda x : get_lyrics(x["genius_artist"], x["genius_track"]), axis=1)

ConnectionError: (MaxRetryError("HTTPSConnectionPool(host='genius.com', port=443): Max retries exceeded with url: /Slim-thug-its-that-p.j.-lyrics (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001CECDBDF320>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))",), 'occurred at index 20001')

In [ ]:
import pickle

with open('songs7_lyrics', 'wb') as fp:
    pickle.dump(songs2, fp)